# Imports

In [1]:
import itertools
import numpy as np
import pandas as pd
import networkx as nx

# Config

In [ ]:
# board size = length of one side of hexagonal board
board_size = 4

# Create Board

* hexagonal 2d space can be represented with 3 axes on a 2d plane equally 
separated
* each integer coordinate is a single square on the board
* neighbours are then +1 in each axes, maxed at the size of the board

In [81]:
nodes = np.array(
    list(itertools.product(*[range(-board_size + 1, board_size)] * 2))
)
directions = list(itertools.product(*[[-1, 0, 1]] * 2))
directions.pop(directions.index((0, 0)))
directions = np.array(directions)

In [82]:
edges = {}
for i, node in enumerate(nodes):
    edges[i] = []
    for direction in directions:
        neighbour = node + direction
        if (np.abs(neighbour) >= board_size).sum() > 0:
            continue
        neighbour_index = nodes.tolist().index(neighbour.tolist())
        edges[i].append(neighbour_index)
g = nx.from_dict_of_lists(edges)

In [88]:
print(f"from node:\n{nodes[0]}\n")
print("to nodes:")
for to_node in g[0].keys():
    print(nodes[to_node])

from node:
[-3 -3]

to nodes:
[-3 -2]
[-2 -3]
[-2 -2]


In [77]:
nodes[0]

array([-3, -3, -3])

In [78]:
4 + 5 + 6 + 7 + 6 + 5 + 4

37

In [84]:
len(nodes)

49

---

In [5]:
def hexagonal_coordinates(n):
    """Generate all coordinates for a hexagonal board with side length n."""
    for x in range(-n + 1, n):
        for y in range(max(-n + 1, -x - n + 1), min(n, -x + n)):
            z = -x - y
            yield (x, y, z)


def hexagonal_neighbors(x, y, z):
    """Yield the neighbors of a given hexagon cell in cube coordinates."""
    # Six possible directions for a hex cell
    directions = [
        (+1, -1, 0),
        (+1, 0, -1),
        (0, +1, -1),
        (-1, +1, 0),
        (-1, 0, +1),
        (0, -1, +1),
    ]
    for dx, dy, dz in directions:
        yield (x + dx, y + dy, z + dz)


def create_hexagonal_board(n):
    """Create a DataFrame representing the edges of a hexagonal board."""
    coords = list(hexagonal_coordinates(n))
    edges = []
    seen = set()

    for x, y, z in coords:
        for nx, ny, nz in hexagonal_neighbors(x, y, z):
            if (nx, ny, nz) in coords:
                edge = tuple(sorted([(x, y, z), (nx, ny, nz)]))
                if edge not in seen:
                    seen.add(edge)
                    edges.append(edge)

    # Creating DataFrame from edges list
    edge_df = pd.DataFrame(edges, columns=["source", "target"])
    return edge_df

In [6]:
# Example usage
n = 2  # Side length of the hexagonal board
df_edges = create_hexagonal_board(n)
print(df_edges)

        source      target
0   (-1, 0, 1)  (0, -1, 1)
1   (-1, 0, 1)   (0, 0, 0)
2   (-1, 0, 1)  (-1, 1, 0)
3   (-1, 1, 0)   (0, 0, 0)
4   (-1, 1, 0)  (0, 1, -1)
5   (0, -1, 1)  (1, -1, 0)
6   (0, -1, 1)   (0, 0, 0)
7    (0, 0, 0)  (1, -1, 0)
8    (0, 0, 0)  (1, 0, -1)
9    (0, 0, 0)  (0, 1, -1)
10  (0, 1, -1)  (1, 0, -1)
11  (1, -1, 0)  (1, 0, -1)


In [7]:
g = nx.from_pandas_edgelist(df_edges)

In [8]:
list(g.nodes())

[(-1, 0, 1),
 (0, -1, 1),
 (0, 0, 0),
 (-1, 1, 0),
 (0, 1, -1),
 (1, -1, 0),
 (1, 0, -1)]

In [9]:
"""
.|0|0|.
|0|0|0|
.|0|0|.
"""

'\n.|0|0|.\n|0|0|0|\n.|0|0|.\n'

In [10]:
"""
(-1, 0, 1)  ->  (-1,1)
(0, -1, 1)  ->  (1,1)
(0, 0, 0)   ->  (0,0)
(-1, 1, 0)  ->  (0,-1)
(0, 1, -1)  ->  (-1,-1)
(1, -1, 0)  ->  (0,1)
(1, 0, -1)  ->  (1,-1)
"""

'\n(-1, 0, 1)  ->  (-1,1)\n(0, -1, 1)  ->  (1,1)\n(0, 0, 0)   ->  (0,0)\n(-1, 1, 0)  ->  (0,-1)\n(0, 1, -1)  ->  (-1,-1)\n(1, -1, 0)  ->  (0,1)\n(1, 0, -1)  ->  (1,-1)\n'

In [16]:
def cubic_to_pixel(x, y, z):
    return (x - y, -x - y)

def pixel_to_array(x, y, n):
    return x+n, y+n

In [24]:
pixel_coordinates = list(map(lambda x: cubic_to_pixel(*x), g.nodes()))
array_coordinates = list(map(lambda x: pixel_to_array(*x, n), pixel_coordinates))
for cn, pn, an in zip(g.nodes(), pixel_coordinates, array_coordinates):
    print(f"{cn}\t->\t{pn}\t->\t{an}")

(-1, 0, 1)	->	(-1, 1)	->	(1, 3)
(0, -1, 1)	->	(1, 1)	->	(3, 3)
(0, 0, 0)	->	(0, 0)	->	(2, 2)
(-1, 1, 0)	->	(-2, 0)	->	(0, 2)
(0, 1, -1)	->	(-1, -1)	->	(1, 1)
(1, -1, 0)	->	(2, 0)	->	(4, 2)
(1, 0, -1)	->	(1, -1)	->	(3, 1)


In [41]:
def print_board(graph, size):
    cubic_nodes = graph.nodes()
    pixel_nodes = list(map(lambda x: cubic_to_pixel(*x), cubic_nodes))
    array_nodes = list(map(lambda x: pixel_to_array(*x, n), pixel_nodes))

#     sorted_pixel_nodes = sorted(
#         pixel_nodes, key=lambda x: (x[1], -x[0]), reverse=True
#     )

    board = np.full((size*2+1, size*2+1), fill_value=".")

    for x,y in array_nodes:
        board[x,y-1] = "|"
        board[x,y+1] = "|"
        board[x,y] = 0

        
    # print board
    for row in board:
        for val in row:
            print(val, end=" ")
        print()
#     print(board)

print_board(g, 2)

. | 0 | . 
| 0 | 0 | 
. | 0 | . 
| 0 | 0 | 
. | 0 | . 


In [123]:
class HexCellNode(nx.Graph):
    default_cell_dict = {
        "count": 0,
        "player": 0
    }
    def init_node_attr(self):
        return self.default_cell_dict
    node_attr_dict_factory = init_node_attr

class Hexplode():
    def __init__(self, size=2):
        self.size = size
        self.board_graph = self.initialise_board_graph()
        self.players = ["Player 1", "Player 2"]
        self.current_player = 0

    @staticmethod
    def hexagonal_coordinates():
        """Generate all coordinates for a hexagonal board with side length n."""
        n = self.size
        for x in range(-n + 1, n):
            for y in range(max(-n + 1, -x - n + 1), min(n, -x + n)):
                z = -x - y
                yield (x, y, z)

    @staticmethod
    def hexagonal_neighbors(x, y, z):
        """Yield the neighbors of a given hexagon cell in cube coordinates."""
        # Six possible directions for a hex cell
        directions = [
            (+1, -1, 0),
            (+1, 0, -1),
            (0, +1, -1),
            (-1, +1, 0),
            (-1, 0, +1),
            (0, -1, +1),
        ]
        for dx, dy, dz in directions:
            yield (x + dx, y + dy, z + dz)

    @staticmethod
    def create_hexagonal_board():
        """Create a DataFrame representing the edges of a hexagonal board."""
        size = self.size
        coords = list(hexagonal_coordinates(size))
        edges = []
        seen = set()

        for x, y, z in coords:
            for nx, ny, nz in hexagonal_neighbors(x, y, z):
                if (nx, ny, nz) in coords:
                    edge = tuple(sorted([(x, y, z), (nx, ny, nz)]))
                    if edge not in seen:
                        seen.add(edge)
                        edges.append(edge)

        # Creating DataFrame from edges list
        edge_df = pd.DataFrame(edges, columns=["source", "target"])
        return edge_df

    @staticmethod
    def cubic_to_pixel(x, y, z):
        return (x - y, -x - y)

    @staticmethod
    def pixel_to_array(x, y, n):
        return x+n, y+n
    
    def initialise_board_graph(self):
        size = self.size
        df_edges = create_hexagonal_board(size)
        g = nx.from_pandas_edgelist(
            df_edges,
            create_using=HexCellNode
        )
        return g
    
    def display_board(self):
        g = self.board_graph
        size = self.size
        cubic_nodes = g.nodes()
        pixel_nodes = list(map(lambda x: cubic_to_pixel(*x), cubic_nodes))
        array_nodes = list(map(lambda x: pixel_to_array(*x, n), pixel_nodes))

        board = np.full((size*2+1, size*2+1), fill_value=".")
        for x,y in array_nodes:
            board[x,y-1] = "|"
            board[x,y+1] = "|"
            board[x,y] = 0

        for row in board:
            for val in row:
                print(val, end=" ")
            print()


    def explode(self, cubic_node):
        g = self.board_graph
        neighbours = g[cubic_node]
        for node in neighbours:
            g.nodes(data=True)[node]["count"] += 1
            g.nodes(data=True)[node]["player"] = self.players[self.current_player]
            if g.nodes(data=True)[node]["count"] > len(neighbours):
                self.explode(node)

    def valid_move(self, node, player):
        g = self.board_graph
        is_valid_node = node in g.nodes()
        if not is_valid_node:
            return 0
        node_player = g.nodes(data=True)[node]["player"]
        is_valid_player = (
            (node_player is None)
            or (node_player == player)
        )
        if not is_valid_player:
            return 0
        return 1

    def make_move(self, node):
        player = self.players[self.current_player]
        if self.valid_move(node, )

SyntaxError: invalid syntax (4251447118.py, line 102)

In [100]:
game = Hexplode()

In [102]:
game.display_board()

. | 0 | . 
| 0 | 0 | 
. | 0 | . 
| 0 | 0 | 
. | 0 | . 


In [121]:
gr.nodes(data=True)[(0,0,0)]["count"] = 2

In [122]:
gr.nodes(data=True)[(0,0,0)]

{'count': 2}